<a href="https://colab.research.google.com/github/tkdlqh2/wanted_8/blob/main/3_NLU_%EB%AC%B8%EC%9E%A5%EC%9C%A0%EC%82%AC%EB%8F%84%EA%B3%84%EC%82%B0/NLU_klue_roberta_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. 설정 및 모듈 임포트

In [ ]:
!pip install transformers datasets wandb

In [2]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine

In [3]:
!pip install hf-lfs
!git config --global user.email "honeybeat1@gmail.com"
!git config --global user.name "lanaChung"

     |████████████████████████████████| 4.0 MB 2.0 MB/s 


In [4]:
import transformers
print(transformers.__version__)

4.17.0


In [42]:
import os
import sys
import pandas as pd
import numpy as np 

import re
import requests
import gc

import torch
import torch.nn as nn
import tensorflow as tf

import math
import logging
from datetime import datetime

from datasets import Dataset

In [6]:
# GPU 메모리 에러 체크
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [7]:
# seed
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla T4
cuda


In [9]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [176]:
print(gc.collect())
torch.cuda.empty_cache()

265


### 1. 데이터셋 불러오기

In [20]:
# 전처리된 데이터 불러오기
train_df = pd.read_csv('/content/drive/MyDrive/data/klue_sts_preprocessed.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/data/klue_sts_preprocessed_dev.csv')

In [11]:
from datasets import load_dataset
dataset = load_dataset('klue', task)

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [106]:
# argument setting
task = "sts"
model_checkpoint = "klue/roberta-large" # 사전학습 언어모델
batch_size = 256 # 높게 

### dataset 불러오는 경우 pass

In [75]:
params = ['guid','khaii1','khaii2','label']

In [76]:
dataset = Dataset.from_pandas(train_df[params])
test_dataset = Dataset.from_pandas(dev_df[params])

In [77]:
dataset, test_dataset

(Dataset({
     features: ['guid', 'khaii1', 'khaii2', 'label'],
     num_rows: 11661
 }), Dataset({
     features: ['guid', 'khaii1', 'khaii2', 'label'],
     num_rows: 519
 }))

In [78]:
dataset = dataset.rename_column('khaii1','sentencel')
dataset = dataset.rename_column('khaii2','sentence2')

In [79]:
dataset

Dataset({
    features: ['guid', 'sentencel', 'sentence2', 'label'],
    num_rows: 11661
})

### 2. 토큰화

In [110]:
# klue/roberta-base 모델의 토크나이저 
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

loading file https://huggingface.co/klue/roberta-base/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/e8441a174492958462b6b16b6db8f1e7253cd149ca779522cadd812d55091b89.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/233a5b2c17873a8477b62dd92a02092a9937759e924a5f22b111becebb8aba5e.44c30ade4958fcfd446e66025e10a5b380cdd0bbe9b3fb7a794f357e7f0f34c2
loading file https://huggingface.co/klue/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/klue/roberta-base/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/9d0c87e44b00acfbfbae931b2e4068eb6311a0c3e71e23e5400bdf57cab4bfbf.70c17d6e4d492c8f24f5bb97ab56c7f272e947112c6faf9dd846da42ba13eb23
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokenizer_config.json from cache 

In [109]:
# train을 train/val로 나누고, val을 score용 test로 쓴다. 
# 9:1로 train/val 나누기
train_dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)

In [111]:
train_dt = train_dataset['train']
val_dt = train_dataset['test']
val_dt

Dataset({
    features: ['guid', 'sentencel', 'sentence2', 'label'],
    num_rows: 1167
})

In [112]:
len(train_dt), len(val_dt)

(10494, 1167)

In [113]:
max_length=512

### tokenizer

In [114]:
# 형태소 전처리 안한 버젼
def preprocess_function(examples):
    return tokenizer(examples['sentencel'], examples['sentence2'],
                     truncation=True, max_length=512, padding=True)

encoded_train = train_dt.map(preprocess_function)
encoded_val = val_dt.map(preprocess_function)

  0%|          | 0/10494 [00:00<?, ?ex/s]

  0%|          | 0/1167 [00:00<?, ?ex/s]

### 2. Fine-tuning

In [115]:
metric_name = "pearsonr"
model_checkpoint

'klue/roberta-base'

In [ ]:
# pretrain 모델의 체크포인트를 다운로드하여 STS Task 학습
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 1
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           num_labels=num_labels)

In [116]:
model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "sts")
logging_dir = os.path.join(output_dir, 'logs')
args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,
    overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "epoch", # 'steps'
    load_best_model_at_end=True,
    # save_steps = 500,


    # Dataset
    num_train_epochs=15,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    
    # Optimizer
    learning_rate=2e-5, # 5e-5
    weight_decay=0.01,  # 0
   # warmup_steps=200,

    # Regularization
    # max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,


    # Evaluation 
    metric_for_best_model='eval_' + metric_name,
    evaluation_strategy = "epoch",

    # HuggingFace Hub Upload
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",

    # Logging
    logging_dir=logging_dir,
    report_to='wandb',

    # Randomness
    seed=42,

    # Mixed-Precision Training
    fp16=True,
    fp16_opt_level='01',
)

PyTorch: setting up devices
/usr/local/lib/python3.7/dist-packages/transformers/training_args.py:952: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case rurupang/roberta-base-finetuned-sts).
  FutureWarning,


In [92]:
from datasets import load_metric
metric_name = "pearsonr"
metric_pearsonr = load_metric(metric_name)

In [93]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    pr = metric_pearsonr.compute(predictions=predictions,
                                  references=labels)
    return pr

In [94]:
from transformers import DataCollatorWithPadding

# Dynamic padding 
data_collator = DataCollatorWithPadding(tokenizer)

In [118]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    data_collator=data_collator, #dynamic padding
    compute_metrics=compute_metrics,
)

/content/test-klue/sts is already a clone of https://huggingface.co/rurupang/roberta-base-finetuned-sts. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
encoded_train[0]

In [119]:
import wandb
wandb.login()

True

In [120]:
id = wandb.util.generate_id()
print(id)

3uxjmiaq


In [121]:
wandb.init(project='klue-sts-roberta',
           entity='honeybeat1',
           id=id,
           )

In [122]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentencel, guid. If sentence2, sentencel, guid are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10494
  Num Epochs = 15
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2460
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Pearsonr
1,No log,0.229864,0.943087
2,No log,0.305525,0.940506
3,No log,0.217434,0.940797
4,0.119200,0.219890,0.940451
5,0.119200,0.218011,0.946489
6,0.119200,0.245017,0.949528
7,0.093300,0.258761,0.942715
8,0.093300,0.242114,0.949352
9,0.093300,0.195656,0.950228
10,0.049000,0.218626,0.948083


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentencel, guid. If sentence2, sentencel, guid are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 64
Saving model checkpoint to test-klue/sts/checkpoint-164
Configuration saved in test-klue/sts/checkpoint-164/config.json
Model weights saved in test-klue/sts/checkpoint-164/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentencel, guid. If sentence2, sentencel, guid are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 64
Saving model checkpoint to test-klue/sts/checkpoin

TrainOutput(global_step=2460, training_loss=0.06108478065428695, metrics={'train_runtime': 2303.7343, 'train_samples_per_second': 68.328, 'train_steps_per_second': 1.068, 'total_flos': 5913469624060068.0, 'train_loss': 0.06108478065428695, 'epoch': 15.0})

In [123]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentencel, guid. If sentence2, sentencel, guid are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 64


{'epoch': 15.0,
 'eval_loss': 0.19565603137016296,
 'eval_pearsonr': 0.9502276369476443,
 'eval_runtime': 4.8356,
 'eval_samples_per_second': 241.336,
 'eval_steps_per_second': 3.929}

In [124]:
wandb.finish()

eval/loss,▃█▂▃▂▄▅▄▁▂▂▃▁▂▂▁
eval/pearsonr,▃▁▁▁▅▇▃▇█▆▇▆▇▇▇█
eval/runtime,▅▂▅▅▅▆▆▇▄▅█▆█▆▆▁
eval/samples_per_second,▄▇▄▄▄▃▃▂▅▄▁▃▁▃▃█
eval/steps_per_second,▄▇▄▄▄▃▃▂▅▄▁▃▁▃▃█
train/epoch,▁▁▂▂▃▃▃▄▄▅▅▅▅▆▇▇▇▇███
train/global_step,▁▁▂▂▃▃▃▄▄▅▅▅▅▆▇▇▇▇███
train/learning_rate,█▆▃▁
train/loss,█▆▃▁
train/total_flos,▁
train/train_loss,▁


In [125]:
trainer.push_to_hub()

Saving model checkpoint to test-klue/sts
Configuration saved in test-klue/sts/config.json
Model weights saved in test-klue/sts/pytorch_model.bin
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
To https://huggingface.co/rurupang/roberta-base-finetuned-sts
   68b19cb..2fddcdf  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Pearsonr', 'type': 'pearsonr', 'value': 0.9502276369476443}]}
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified
remote: tput: No value for $TERM and no -T specified
remote: tput: No value for $TERM and no -T specified        
To https://huggingface.co/rurupang/roberta-base-finet

'https://huggingface.co/rurupang/roberta-base-finetuned-sts/commit/2fddcdf5f264fd3bd7720f28239f745af6d1be4f'

### Predict

In [126]:
test_dataset

Dataset({
    features: ['guid', 'sentencel', 'sentence2', 'label'],
    num_rows: 519
})

In [ ]:
#test_dataset = test_dataset.rename_column('khaii1','sentencel')
#test_dataset = test_dataset.rename_column('khaii2','sentence2')

In [128]:
encoded_test = test_dataset.map(preprocess_function)

  0%|          | 0/519 [00:00<?, ?ex/s]

In [129]:
result = trainer.predict(encoded_test)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentencel, guid. If sentence2, sentencel, guid are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 519
  Batch size = 64


In [130]:
result[2]

{'test_loss': 0.42094993591308594,
 'test_pearsonr': 0.8537968863939722,
 'test_runtime': 2.091,
 'test_samples_per_second': 248.202,
 'test_steps_per_second': 4.304}

--- 여기까지 ---

In [105]:
# fine-tune model pearsonr : 0.892
result[2]

{'test_loss': 0.45634862780570984,
 'test_pearsonr': 0.8553856922399568,
 'test_runtime': 2.03,
 'test_samples_per_second': 255.667,
 'test_steps_per_second': 8.374}

### Hyperparameter Search

In [ ]:
!pip install optuna

In [161]:
model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "sts-hps")
logging_dir = os.path.join(output_dir, 'logs')

args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,
    # overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "epoch", # 'steps'
    load_best_model_at_end=True,
    # save_steps = 500,


    # Dataset
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    
    # Optimizer
    learning_rate=2e-5, # 5e-5
    weight_decay=0.01,  # 0
    warmup_steps=200,

    # Resularization
    # max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,


    # Evaluation 
    metric_for_best_model='eval_pearsonr',
    evaluation_strategy = "epoch",

    # HuggingFace Hub Upload
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",

    # Logging
    logging_dir=logging_dir,
    report_to='wandb',

    # Randomness
    seed=42,
        
    # Mixed-Precision Training
    fp16=True,
    fp16_opt_level='01',
)

PyTorch: setting up devices
/usr/local/lib/python3.7/dist-packages/transformers/training_args.py:952: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case rurupang/roberta-base-finetuned-sts).
  FutureWarning,


In [160]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [177]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_train.shard(index=1, num_shards=10),
    eval_dataset=encoded_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

loading configuration file https://huggingface.co/klue/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "

In [ ]:
wandb.login()

True

In [ ]:
id = wandb.util.generate_id()
print(id)

pt0a6kxq


In [ ]:
wandb.init(project='klue-sts-roberta-base',
           entity='honeybeat1',
           id=id)

In [180]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-6, 5e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
        "weight_decay":  trial.suggest_float("weight_decay", 1e-6, 1e-1),
        'warmup_steps': trial.suggest_int('warmup_steps', 0, 500),
    }

In [181]:
def objective(metrics):
    return metrics["eval_pearsonr"]

In [182]:
best_run = trainer.hyperparameter_search(n_trials=5, 
                                         direction="maximize", 
                                         hp_space=optuna_hp_space, 
                                         compute_objective=objective)

[I 2022-03-22 20:10:07,808] A new study created in memory with name: no-name-f6c1d127-187c-4dc5-8ee1-17a49d0e9b0b
Trial:
loading configuration file https://huggingface.co/klue/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hi

eval/loss,█▅▁
eval/pearsonr,▁
eval/runtime,▁█▂
eval/samples_per_second,█▁▇
eval/steps_per_second,█▁▇
train/epoch,▁▅██
train/global_step,▁▅██
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Pearsonr
1,No log,3.084072,nan
2,No log,3.071100,nan


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentencel, guid. If sentence2, sentencel, guid are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 64
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
Saving model checkpoint to test-klue/sts-hps/run-0/checkpoint-132
Configuration saved in test-klue/sts-hps/run-0/checkpoint-132/config.json
Model weights saved in test-klue/sts-hps/run-0/checkpoint-132/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentencel, guid. If sent

...

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentencel, guid. If sentence2, sentencel, guid are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 64
Saving model checkpoint to test-klue/sts-hps/run-3/checkpoint-263
Configuration saved in test-klue/sts-hps/run-3/checkpoint-263/config.json
Model weights saved in test-klue/sts-hps/run-3/checkpoint-263/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentencel, guid. If sentence2, sentencel, guid are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 64
Saving model checkpo

eval/loss,█▆▁▂
eval/pearsonr,▁▆██
eval/runtime,▆█▁▂
eval/samples_per_second,▃▁█▇
eval/steps_per_second,▃▁█▇
train/epoch,▁▃▃▆███
train/global_step,▁▃▃▆███
train/learning_rate,█▁
train/loss,█▁
train/total_flos,▁
train/train_loss,▁


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Pearsonr
1,No log,2.979069,nan
2,No log,0.641636,0.871308
3,No log,0.388899,0.933582
4,2.092300,0.371645,0.933896


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentencel, guid. If sentence2, sentencel, guid are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 64
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
Saving model checkpoint to test-klue/sts-hps/run-4/checkpoint-132
Configuration saved in test-klue/sts-hps/run-4/checkpoint-132/config.json
Model weights saved in test-klue/sts-hps/run-4/checkpoint-132/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, sentencel, guid. If sent

In [183]:
best_run

BestRun(run_id='3', objective=0.9343047314351852, hyperparameters={'learning_rate': 1.014390700275144e-05, 'num_train_epochs': 4, 'per_device_train_batch_size': 4, 'weight_decay': 0.053536718175078706, 'warmup_steps': 126})

In [184]:
best_run.hyperparameters

{'learning_rate': 1.014390700275144e-05,
 'num_train_epochs': 4,
 'per_device_train_batch_size': 4,
 'warmup_steps': 126,
 'weight_decay': 0.053536718175078706}

### 5. Hyperparameter 적용해보기

In [ ]:
model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "sts-hps")
logging_dir = os.path.join(output_dir, 'logs')

args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,
    # overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "epoch", # 'steps'
    load_best_model_at_end=True,
    # save_steps = 500,


    # Dataset
    num_train_epochs=best_run.hyperparameters['num_train_epochs'],
    per_device_train_batch_size=best_run.hyperparameters['per_device_train_batch_size'],
    per_device_eval_batch_size=best_run.hyperparameters['per_device_train_batch_size'],
    
    # Optimizer
    learning_rate=best_run.hyperparameters['learning_rate'],
    weight_decay=best_run.hyperparameters['weight_decay'],
    warmup_steps=best_run.hyperparameters['warmup_steps'],

    # Resularization
    # max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,


    # Evaluation 
    metric_for_best_model='eval_pearsonr',
    evaluation_strategy = "epoch",

    # HuggingFace Hub Upload
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",

    # Logging
    logging_dir=logging_dir,
    report_to='wandb',

    # Randomness
    seed=42,
)

PyTorch: setting up devices
/usr/local/lib/python3.7/dist-packages/transformers/training_args.py:952: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case rurupang/roberta-base-finetuned-sts).
  FutureWarning,


In [ ]:
trainer = Trainer(
    model,
    args=args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/content/test-klue/sts-hps is already a clone of https://huggingface.co/rurupang/roberta-base-finetuned-sts. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
id = wandb.util.generate_id()
print(id)
wandb.init(project='klue-sts-roberta',
           entity='honeybeat1',
           id=id,
           )

1r5mhw3h


eval/loss,▄█▁
eval/runtime,▄▁█
eval/samples_per_second,▅█▁
eval/steps_per_second,▅█▁
train/epoch,▁▅██
train/global_step,▁▅██
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, labels.binary-label, guid, labels.label, source, sentence1. If sentence2, labels.binary-label, guid, labels.label, source, sentence1 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10501
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6565
Automatic Weights & Biases logging enabled, to di

Epoch,Training Loss,Validation Loss,Pearsonr
1,0.462400,0.307781,0.922406
2,0.382000,0.404757,0.934564
3,0.293600,0.253247,0.939978
4,0.176100,0.221119,0.941076
5,0.101700,0.203461,0.947305


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, labels.binary-label, guid, labels.label, source, sentence1. If sentence2, labels.binary-label, guid, labels.label, source, sentence1 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 8
Saving model checkpoint to test-klue/sts-hps/checkpoint-1313
Configuration saved in test-klue/sts-hps/checkpoint-1313/config.json
Model weights saved in test-klue/sts-hps/checkpoint-1313/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence2, labels.binary-label, guid, labels.label, source, sentence1. If sentence2, labels.binary-label, guid, labels.label, source, sentence1 are not expected by `RobertaForSeque

In [ ]:
trainer.evaluate()

In [ ]:
wandb.finish()

In [ ]:
result = trainer.predict(encoded_test)
result[2]